In [1]:
import tensorflow as tf
import numpy as np
import vgg16_network as net
import read_img
import pandas as pd
import os

In [2]:
#选4个司机做交叉验证，正则化增加到0.0002
checkpoint_dir = '/path/to/save_model/Driver_Detection_ver8/'
ckt_name = 'model.ckpt-32288'

#0.589
#checkpoint_dir = '/path/to/save_model/Driver_Detection_ver3/'
#ckt_name = 'model.ckpt-20609'

#0.617
#checkpoint_dir = '/path/to/save_model/Driver_Detection_ver3/'
#ckt_name = 'model.ckpt-27777'

#1.274
##checkpoint_dir = '/path/to/save_model/Driver_Detection_ver1/'
##ckt_name = 'model.ckpt-29009'

In [3]:
def test(dataAug=False):
    test_imgs_list = pd.read_csv('data/sample_submission.csv')
    input_data = tf.placeholder(tf.float32,shape=[1,None,None,3],name='img_input')
    if dataAug:
        print("test datas use data aug")
        vgg_input = read_img.tfDataAug(input_data,batch_size=1)
    else:
        vgg_input = input_data
    cls_10 = net.VGG16_test(vgg_input,checkpoint_dir,ckt_name)
    cls_10_softmax = tf.nn.softmax(cls_10)
    ########################################
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    result =[]
    with tf.Session(config=config) as sess:
        tf.global_variables_initializer().run()
        #for i in range(10):
        for i in range(len(test_imgs_list.img)):
            img_file = os.path.join('data','test\\') + test_imgs_list.img[i]
            img = read_img.get_img_sample(img_file,dataAug=dataAug)
            if((i+1)%5000==0): print("finish",i)
            pre_cls = sess.run(cls_10_softmax,feed_dict={input_data:img})
            pre_cls = np.clip(pre_cls,10**-15,1-10**-15)
            tmp = [test_imgs_list.img[i],pre_cls[0].tolist()]
            result.append(tmp)
    return result,pre_cls

def cross_val(file_name):
    val_imgs_list = pd.read_csv(file_name)
    input_data = tf.placeholder(tf.float32,shape=[1,None,None,3],name='img_input')
    cls_10 = net.VGG16_test(input_data,checkpoint_dir,ckt_name)
    cls_10_softmax = tf.nn.softmax(cls_10)
    ########################################
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    error =0
    data_num = len(val_imgs_list.img)
    with tf.Session(config=config) as sess:
        tf.global_variables_initializer().run()
        for i in range(len(val_imgs_list.img)):
            img_file = os.path.join('data','train', val_imgs_list.classname[i]) + '\\'+ val_imgs_list.img[i]
            img = read_img.get_img_sample(img_file)
            pre_cls = sess.run(cls_10_softmax,feed_dict={input_data:img})
            if np.argmax(pre_cls)!=int(val_imgs_list.classname[i][1]): 
                error+=1
    return error,data_num

def get_sumbcsv(result,filename):
    test = [result[i][1] for i in range(len(result))]
    name = [result[i][0] for i in range(len(result))]
    c0 = [test[i][0] for i in range(len(test))]
    c1 = [test[i][1] for i in range(len(test))]
    c2 = [test[i][2] for i in range(len(test))]
    c3 = [test[i][3] for i in range(len(test))]
    c4 = [test[i][4] for i in range(len(test))]
    c5 = [test[i][5] for i in range(len(test))]
    c6 = [test[i][6] for i in range(len(test))]
    c7 = [test[i][7] for i in range(len(test))]
    c8 = [test[i][8] for i in range(len(test))]
    c9 = [test[i][9] for i in range(len(test))]
    dataframe = pd.DataFrame({'img':name,'c0':c0,'c1':c1,'c2':c2,'c3':c3,'c4':c4,'c5':c5,'c6':c6,'c7':c7,'c8':c8,'c9':c9})
    dataframe.to_csv(filename,index=False,sep=',')

In [4]:
res,pre_cls = test(dataAug=False)
get_sumbcsv(res,'9_VGG16_l2_flip_mirror_rota90.csv')

finish 4999
finish 9999
finish 14999
finish 19999
finish 24999
finish 29999
finish 34999
finish 39999
finish 44999
finish 49999
finish 54999
finish 59999
finish 64999
finish 69999
finish 74999


In [9]:
error,data_num = cross_val('cross_validation11.csv')
print(error,data_num)

In [2]:
driver1 = pd.read_csv('submisson/1_VGG16_cro98.csv')
driver2 = pd.read_csv('submisson/5_VGG16_loss0.07.csv')
driver3 = pd.read_csv('submisson/3_VGG16_cro98_dataAug.csv')
driver4 = pd.read_csv('submisson/7_VGG16_l2_0002_subject.csv')
driver = (driver1.values[:,:10] + driver2.values[:,:10] + driver3.values[:,:10]+ driver4.values[:,:10])/4
name = driver1.values[:,10]
c0 = driver[:,0]
c1 = driver[:,1]
c2 = driver[:,2]
c3 = driver[:,3]
c4 = driver[:,4]
c5 = driver[:,5]
c6 = driver[:,6]
c7 = driver[:,7]
c8 = driver[:,8]
c9 = driver[:,9]
dataframe = pd.DataFrame({'img':name,'c0':c0,'c1':c1,'c2':c2,'c3':c3,'c4':c4,'c5':c5,'c6':c6,'c7':c7,'c8':c8,'c9':c9})
dataframe.to_csv('8_VGG_test.csv',index=False,sep=',')